# **Solving the Vehicle Routing Problem**
*The Qubit Players*
- QOSF Quantum Computing Mentorship Program 2021 Cohort - 4
- Mentor: Dr. Vesselin G. Gueorguiev
- Mentees: Arya Bhatta, Asish Kumar Mandoi

## The Vehicle Routing Problem (VRP)
- A combinatorial optimization and integer programming problem
- Problem statement: ***What is the optimal set of routes for a fleet of vehicles to traverse in order to deliver to a given set of customers?***
- Generalises **The Travelling Salesman Problem (TSP)**.
- An NP-hard problem.

## How are we solving it?
- Model the optimization problem using the QuadraticProgram class in `Qiskit`'s Optimization module
- Convert this QuadraticProgram into a QUBO problem
- Feed this QUBO problem to one of the Quantum Annealers provided by D-Wave
- Multiple methods to solve the VRP
- We specifically focussed on one particular method: **RAS**

## Route Activation Solver (RAS)

Explored multiple ways to make this solver work perfectly

### Approach 1:

**Minimize:**
$$C = \sum_{i=0}^N \sum_{j=0,\ j \neq i}^N C_{ij} x_{ij}$$

where $x_{ij}$ is a binary descision varible and $C_{ij}$ is the cost (or weight) associated with the edge connecting node $i$ and node $j$

$$x_{ij} = 
\left\{\begin{matrix}
1; & if\ there\ exists\ a\ path\ from\ the\ i^{th}\ node\ to\ the\ j^{th}\ node\\
0; & otherwise.
\end{matrix}\right.$$

**Subject to the following constraints**

1. a) Each node other than the depot has exactly $1$ outgoing active edge and exactly $1$ incoming active edge.

    $$\sum_{j=0,\ j\neq i}^N x_{ij} = 1, \sum_{j=0,\ j\neq i}^N x_{ji} = 1, \ \forall i$$

1. b) Each node is visited by exactly one vehicle

    $$\sum_{i=1}^N y_{ik} = 1; \;\; \forall k \in \{1,2,\dots,M\}$$
  
    where $y_{ik}$ is another binary descision variable,

    $$y_{ik} = 
    \left\{\begin{matrix}
    1; & if\ the\ i^{th}\ node\ has\ been\ visited\ by\ the\ k^{th}\ vehicle\\
    0; & otherwise.
    \end{matrix}\right.$$

2. The depot has exactly $M$ outgoing and $M$ incoming connections.
    
    $$\sum_{i=1}^N x_{0i} = M, \;\; \sum_{i=1}^N x_{i0} = M$$

<center><img src="./images/possible_solution_6-2_1.svg" width="640"></center>
<center>Expected</center>

<center><img src="./images/possible_solution_6-2_2.svg" width="640" /></center>
<center>Not expected, but our constraints allow this solution as well</center>

3. The routes must be connected i.e. there must be **no other subroutes** for each vehicle

    Two formulations for this constraint:

    a) DFJ formulation (Dantzig, Fulkerson and Johnson):
      
    $$\sum_{i \in S}\sum_{j \notin S} x_{ij} \ge 1;\;\;\;\; \forall S ⊂ \{1,2,\dots,N\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,M\}\\
    or\ alternatively,\\
    \sum_{i \in S}\sum_{j \in S} x_{ij} \le |S| - 1;\;\;\;\; \forall S ⊂ \{1,2,\dots,N\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,M\}$$
      
    b) MTZ formulation (Miller, Tucker and Zemlin, and subsequently extended by Christofides, Mingozzi and Toth)
      
    $$t_j \cdot y_{jk} \ge (t_i + 1) \cdot y_{ik} - B(1 - x_{ij});\;\;\;\; \forall k \in {1,2,\dots,M}$$
    where $t_i$ is the time taken for any vehicle to reach node $i$, and $B$ is a large number
      
    *Note: Here this MTZ constraint is no longer linear (becomes a quadratic constraint).*

***Qubit Complexity***

  - No. of binary variables:
      - $O(N^2)$ (with quadratic constraints)
      - $O(N^2M)$ (with linear constraints)
  - No. of qubits:
      - $O(N^2)$ (with quadratic constraints)
      - $O(N^2M)$ (with linear constraints)


### Approach 2:
**Minimize:**
$$C = \sum_{k=1}^M \sum_{i=0}^N \sum_{j=0, \ j \neq i}^N C_{ij} x^k_{ij}$$

$$x^k_{ij} = 
\left\{\begin{matrix}
1; & if\ the\ path\ from\ the\ i^{th}\ node\ to\ the\ j^{th}\ node\ is\ traversed\ by\ the\ k^{th}\ vehicle\\
0; & otherwise.
\end{matrix}\right.$$

**Subject to the following constraints**

1. Each node other than the depot has exactly $1$ outgoing active edge and exactly $1$ incoming active edge.

    $$\sum_{k=1}^M \ \sum_{j=0,\ j\neq i}^N x^k_{ij} = 1,\;\;\; \sum_{k=1}^M \ \sum_{j=0,\ j\neq i}^N x^k_{ji} = 1,\;\;\;\;\; \forall i$$

2. The depot has exactly $M$ outgoing and $M$ incoming connections.

    $$\sum_{k=1}^M \sum_{i=1}^N x^k_{0i} = M, \;\; \sum_{k=1}^M \sum_{i=1}^N x^k_{i0} = M$$

3. The routes must be connected i.e. there must be **no other subroutes** for each vehicle

    Again two formulations for this constraint:

    i. DFJ formulation (Dantzig, Fulkerson and Johnson):

    $$\sum_{i \in S}\sum_{j \notin S} x^k_{ij} \ge 1;\;\;\;\; \forall S ⊂ \{1,2,...,n\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,N\}$$
    $$or\ alternatively,$$
    $$\sum_{i \in S}\sum_{j \in S} x^k_{ij} \le |S| - 1;\;\;\;\; \forall S ⊂ \{1,2,...,n\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,N\}$$

    ii. MTZ formulation (Miller, Tucker and Zemlin, and subsequently extended by Christofides, Mingozzi and Toth)

    $$t^k_j \ge t^k_i + 1 - B(1 - x^k_{ij});\;\;\;\; \forall k \in {1,2,\dots,N}$$
    where $t^k_i$ is the time taken for the $k^{th}$ vehicle to reach node $i$, and $B$ is a large number

    *This constraint however is not quadratic.*

***Qubit Complexity***
 - No. of binary variables: $O(N^2M)$
 - No. of qubits: $O(N^2M)$


### Approach 3:

  - Our approach here is to solve the TSP first, i.e. we assume that initially there is only one vehicle. When we find the best route for this one vehicle, we then partion it into multiple subroutes starting and ending at the depot so every subroute is traversed independently by a different vehicle.
  - This partitioning is done classically in such a way that there are M - 1 partitions and the cost of these M - 1 partitions are minimum.

**Minimize:**
$$C = \sum_{i=0}^N \ \sum_{j=0,\ j \neq i}^N C_{ij} x_{ij}$$

$$x_{ij} = 
\left\{\begin{matrix}
1; & if\ there\ exists\ a\ path\ from\ the\ i^{th}\ node\ to\ the\ j^{th}\ node\\
0; & otherwise.
\end{matrix}\right.$$

**Subject to the following constraints**

1. Each node other than the depot has exactly $1$ outgoing active edge and exactly $1$ incoming active edge.

    $$\sum_{j=0,\ j\neq i}^N x_{ij} = 1, \sum_{j=0,\ j\neq i}^N x_{ji} = 1, \ \forall i$$

1. The depot has exactly $M$ outgoing and $M$ incoming connections.
  
    $$\sum_{i=1}^N x_{0i} = M, \;\; \sum_{i=1}^N x_{i0} = M$$

3. The routes must be connected i.e. there must be **no other subroutes** for each vehicle

    Again two formulations for this constraint:

    i. DFJ formulation (Dantzig, Fulkerson and Johnson):
      
    $$\sum_{i \in S}\sum_{j \notin S} x_{ij} \ge 1;\;\;\;\; \forall S ⊂ \{1,2,\dots,N\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,N\}\\
    or\ alternatively,\\
    \sum_{i \in S}\sum_{j \in S} x_{ij} \le |S| - 1;\;\;\;\; \forall S ⊂ \{1,2,\dots,N\},\ |S| \ge 2\;\;\;\; \forall k \in \{1,2,\dots,N\}$$
      
    ii. MTZ formulation (Miller, Tucker and Zemlin, and subsequently extended by Christofides, Mingozzi and Toth)
      
    $$t_j \ge t_i + 1 - B(1 - x_{ij})$$
    where $t_i$ is the time taken for any one vehicle to reach node $i$, and $B$ is a large number

***Qubit Complexity***
 - No. of binary variables: $O(N^2)$
 - No. of qubits: $O(N^2)$


## Conclusion
- We need to be careful about how we formulate the problem, specifically **the number of variables we use**
- The variables are mapped to qubits [[2](https://arxiv.org/pdf/1811.07403.pdf)]; number of variables ~ number of qubits
- The **Solution Partition Solver** (SPS) uses the least number of qubits compared to other methods
- For example: for 5 clients and 2 vehicles, SPS uses only around 25, our best approach of RAS uses around 50

## Next steps
  - Use techniques to achieve better accuracy for larger QUBO problems
  - Implement DBSCAN (a better?) solver
  - Incorporate solutions for other variants of the problem
  - Make a circuit based algorithm from scratch (that does not use any application modules from any libraries) to solve VRP
  - Find potential real life applications for VRP (for e.g. Supply Chain, Traffic Flow Optimization)


## References
[[1](https://github.com/VGGatGitHub/QOSF-cohort3)] The QOSF cohort - 3 project by Shantom, Aniket and Avneesh based on this topic<br>
[[2](https://arxiv.org/pdf/1811.07403.pdf)] Sebastian Feld, et al. "A Hybrid Solution Method for the Capacitated Vehicle Routing Problem Using a Quantum Annealer"<br>
[[3](https://faculty.math.illinois.edu/~mlavrov/slides/482-spring-2020/slides35.pdf)] Misha Lavrov "Travelling Salesman Problem"<br>
[[4](https://en.wikipedia.org/wiki/Vehicle_routing_problem)] Vehicle routing problem - Wikipedia